In [ ]:
import numpy as np
import time
import tensorflow as tf
import kerasncp as kncp
from kerasncp.tf import LTCCell

# from kerasncp.ltc_cell import LTCCell
from kerasncp.wirings import NCP
from keras.utils import plot_model
from keras.models import Sequential
from ncps.tf import LTC

from keras.layers import (
    Lambda,
    Conv2D,
    Conv3D,
    Dropout,
    Dense,
    Flatten,
    Reshape,
    RNN,
    Input,
    Add,
    MaxPool2D,
    MaxPooling3D,
    InputLayer,
    LSTM,
    TimeDistributed,
    Activation,
)
from ncps import wirings
import seaborn as sns
import matplotlib.pyplot as plt

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
NAME = "LTC-CNN3D-Carla-{}".format(int(time.time()))

In [ ]:
tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

In [1]:
from config import GlobalConfig
from data_2D import CARLA_Data
config = GlobalConfig(setting="02_05_withheld")

Skip Town02 and Town05


In [3]:
for epoch in range(10):
    print("Epoch: ", epoch)
    train_set = CARLA_Data(
        root=config.train_data, current_epoch=epoch, config=config
    ).create_dataset()
    val_set = CARLA_Data(
        root=config.val_data, current_epoch=epoch, config=config
    ).create_dataset()

Epoch:  0
Epoch:  1
Epoch:  2
Epoch:  3
Epoch:  4
Epoch:  5
Epoch:  6
Epoch:  7
Epoch:  8
Epoch:  9


In [ ]:
train_set = CARLA_Data(root=config.train_data, config=config).data_generator()

In [4]:
for sample in train_set.take(1).as_numpy_iterator():
    print(sample[0].shape)
    print(sample[1].shape)

kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul
kul


wiring and model based on Convolutions

In [ ]:

"""
Define the Keras sequential CNN-NCP hybrid architecture.

A CNN feature extractor is stacked with an NCP RNN temporal modelling structure.

:return: the CNN-NCP model
"""

# Set the NCP wiring
wiring = NCP(
    inter_neurons=12,  # Number of inter neurons
    command_neurons=8,  # Number of command neurons
    motor_neurons=4,  # Number of motor neurons
    sensory_fanout=4,  # How many outgoing synapses has each sensory neuron
    inter_fanout=4,  # How many outgoing synapses has each inter neuron
    recurrent_command_synapses=4,  # How many recurrent synapses are in the command neuron layer
    motor_fanin=6,  # How many incoming synapses has each motor neuron
)
# Create the the NCP cell based on the LTC neuron.
ncp_cell = LTCCell(wiring)



In [ ]:
wiring = wirings.AutoNCP(12,4)

In [ ]:
model = Sequential()
model.add(Conv3D(32, (3, 3, 3), activation='relu', input_shape=(10, 160, 704, 3), padding='same'))
model.add(Conv3D(32, (3, 3, 3), activation='relu', padding='same'))
model.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same', data_format='channels_first'))
model.add(Dropout(0.25))

model.add(Conv3D(64, (3, 3, 3), activation='relu', padding='same'))
model.add(Conv3D(64, (3, 3, 3), activation='relu', padding='same'))
model.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same', data_format='channels_first'))
model.add(Dropout(0.25))

model.add(TimeDistributed(Flatten()))
#model.add(LSTM(512, return_sequences=True))
model.add(Dense(100, activation='relu'))
model.add(LTC(wiring, return_sequences=True))


model.summary()

In [ ]:
model.compile(
    optimizer=Adam(0.01), loss=MeanSquaredError() , metrics=["mean_squared_error"]
)

In [ ]:
plot_model(model, show_shapes=True)

In [ ]:
hist = model.fit(train_set, epochs=3, validation_data=val_set, callbacks=[tensorboard])

In [ ]:

# Build the sequential hybrid model
model = Sequential(
    [
        Conv2D(24, 5, 2, padding='same', activation="relu", input_shape=(160, 704, 3)),
        Conv2D(36, 5, 2, activation="relu"),
        Conv2D(48, 5, 2, activation="relu"),
        Conv2D(64, 3, activation="relu"),
        Conv2D(64, 3, activation="relu"),
        Dropout(0.5),
        Flatten(),
        Dense(100, activation="relu"),
        Dense(100),
        RNN(ncp_cell, unroll=True),
    ]
)

model.compile(
    optimizer=Adam(0.01), loss=MeanSquaredError() , metrics=["mean_squared_error"]
)

#model.summary()
# plot_model(model, show_shapes=True)
# return model

In [ ]:
model = Sequential([
    Conv3D(32, (3, 3, 3), activation='relu', input_shape=(None, 160, 704, 3), padding='same'),
    Conv3D(32, (3, 3, 3), activation='relu', padding='same'),
    MaxPooling3D(pool_size=(3, 3, 3), padding='same'),
    Dropout(0.25),

    Conv3D(64, (3, 3, 3), activation='relu', padding='same'),
    Conv3D(64, (3, 3, 3), activation='relu', padding='same'),
    MaxPooling3D(pool_size=(3, 3, 3), padding='same'),
    Dropout(0.25),

    Flatten(),
    Dense(512, activation='relu'),
    RNN(ncp_cell, unroll=True),
])

In [ ]:
sns.set_style("white")
plt.figure(figsize=(30, 25))
legend_handles = ncp_cell.draw_graph(layout='shell',neuron_colors={"command": "tab:cyan"})
plt.legend(handles=legend_handles, loc="upper center", bbox_to_anchor=(1, 1))
sns.despine(left=True, bottom=True)
plt.tight_layout()
plt.show()

In [ ]:
ncp_wiring = kncp.wirings.NCP(
    inter_neurons=20,  # Number of inter neurons
    command_neurons=10,  # Number of command neurons
    motor_neurons=5,  # Number of motor neurons
    sensory_fanout=4,  # How many outgoing synapses has each sensory neuron
    inter_fanout=5,  # How many outgoing synapses has each inter neuron
    recurrent_command_synapses=6,  # Now many recurrent synapses are in the
    # command neuron layer
    motor_fanin=4,  # How many incoming synapses has each motor neuron
)
ncp_cell_time = LTCCell(
    ncp_wiring,
    initialization_ranges={
        # Overwrite some of the initialization ranges
        "w": (0.2, 2.0),
    },
)

In [ ]:
time_d_model = Sequential(
    [
        InputLayer(input_shape=(None, 160, 704, 3)),
        TimeDistributed(
            Conv2D(32, (5, 5), activation="relu")
        ),
        TimeDistributed(MaxPool2D()),
        TimeDistributed(
            Conv2D(64, (5, 5), activation="relu")
        ),
        TimeDistributed(MaxPool2D()),
        TimeDistributed(Flatten()),
        TimeDistributed(Dense(32, activation="relu")),
        RNN(ncp_cell_time, return_sequences=True),
        TimeDistributed(Activation("softmax")),
    ]
)
time_d_model.compile(
    optimizer=Adam(0.01),
    loss='sparse_categorical_crossentropy',
)
time_d_model.summary()

In [ ]:
sns.set_style("white")
plt.figure(figsize=(10,10))
legend_handles = ncp_cell_time.draw_graph(layout='spiral',neuron_colors={"command": "tab:cyan"})
plt.legend(handles=legend_handles, loc="upper center", bbox_to_anchor=(1, 1))
sns.despine(left=True, bottom=True)
plt.tight_layout()
plt.show()